Importing all necessary packages

In [13]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain import hub
import tiktoken

Setting OpenAI API Key

In [14]:
import constants
api_key = constants.openai_api_key

Loading the pdf document using PyPDFLoader from langchain

In [15]:
pdf_path = 'crime-and-punishment.pdf'
loader = PyPDFLoader(pdf_path, extract_images=False)

data = []
pages = loader.load()
data.extend(pages[1:])

# words = []
# for i in data: 
#   words.append(i.page_content)
# words = ' '.join(words)
# len(words)



Split the loaded document into chunks

In [16]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=[".", ",", "\n", "\n \n", ";"],
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False
)
chunks = text_splitter.split_documents(data)
len(chunks)


1298

Using RefineDocumentsChain from langchain

In [18]:

llm = OpenAI(temperature = 0.25, openai_api_key=api_key)
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "It's eventually going to be a precise 20-page summary of the complete book"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary. There might be events narrated in the book, take care of that."
    "If the context isn't of great use, do not add anything from it."
)
refine_prompt = PromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)
result = chain({"input_documents": chunks}, return_only_outputs=True).invoke()

c:\Users\javairia.younis\Desktop\PDFSummarizer\.venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************wyUo. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

Prompt Management and Optimization: Langchain offers tools for managing and optimizing prompts, which is crucial for generating accurate and relevant summaries.

Data Augmented Generation: This feature allows Langchain to fetch relevant data from external sources before generating summaries, ensuring that the output is both comprehensive and contextually accurate.

Chains and Agents: Langchain's architecture supports the creation of chains and agents, which can perform a series of LLM calls or actions based on the data retrieved, leading to more dynamic and intelligent summarization processes.